In [1]:
import pandas as pd


In [2]:
imUsingColab = False
shouldDownload = True # Set this to False after the first run to avoid redownloading the dataset
if shouldDownload:
    import downloadData
    downloadData.download(imUsingColab)


Opening zip file
['ratings.csv', 'ratings_small.csv', 'links.csv', 'links_small.csv', 'keywords.csv', 'movies_metadata.csv', 'credits.csv']
Loaded Zip File
Attempting extraction
Extraction Completed
